# Pipeline for Tracking Many Files


##### Luco Buise
##### MSc Thesis: Radboud University

22-05-2025

---

## Imports

In [ ]:
import os

import matplotlib as mpl
import matplotlib.pyplot as plt

# change the following to %matplotlib notebook for interactive plotting
%matplotlib inline

import numpy as np
import pandas as pd
from pandas import DataFrame, Series  # for convenience

import cv2
import trackpy as tp
import ipywidgets as widgets
import pickle
from IPython.display import display
from scipy.optimize import curve_fit
from scipy.signal import savgol_filter

from ipywidgets import HBox, Textarea, interact

---

## Helper Functions

In [ ]:
def detect_circles(image, min_radius, max_radius, param1, param2, dp=1.2):
    min_dist = int(0.9 * max_radius)

    # apply Hough transform
    circles = cv2.HoughCircles(image, cv2.HOUGH_GRADIENT, dp, min_dist,
                               param1=param1, param2=param2,
                               minRadius=min_radius, maxRadius=max_radius)
    return circles
    
def capture_frame(video_obj, frame_num):
    video_obj.set(cv2.CAP_PROP_POS_FRAMES, frame_num)
    ret, frame = video_obj.read()
    if not ret:
        return None # if reached ending, stop
    return frame

def draw_circles(ax, circles):
    if circles is not None:
        for pt in circles[0]:
            x, y, r = pt
            circle = plt.Circle((x, y), r, color='r', fill=False)
            ax.add_patch(circle)

def crop_image(img,x0,y0,x1,y1):
    return img[y0:y1,x0:x1,:]


---

## Hyperparameters

Fill in the parameters found using the $\textbf{tracking\_pipeline\_v2.ipynb}$ notebook.

In [ ]:
# parameters used for all videos (mostly global params)

# ratio between cm and pixels
px_cm_ratio = 0.1905933580903535

# cropping of the video
crop_x = (581, 1378) 
crop_y = (87, 862)

# parameters for bot detection
canny_edge_thresh = 47 # how sharp are the edges considered
circle_detection_thresh = 15 # lower = more sensitive, but can detect false positives.
radiusMin = 10
radiusMax = 19

# what part of the video do you want to track
start_frame = 0
end_frame = 0 # use 0 if end, otherwise use negative number

# parameters for trajectory creation
max_dist = 10 # maximum distance particle can move between frames
memory = 10 # maximum number of frames during which a feature can vanish, and be considered the same particle
min_length = 100 # minimum length for a trajectory

# do you want to smoothen the trajectories
smoothen = True

# do you want to see the plots of the trajectories (as a sanity check)
show_plot = True

# which dir to save all the files
save_dir = "cluster_trajs"

# if dir does not exist, create it
if not os.path.exists(save_dir):
    os.makedirs(save_dir)

## Import videos

Define the folder in which all the files are located

In [ ]:
# folder in which the video folders are located
og_folder = "lab_recordings"

# folder in which all mp4s are located
dir_name = "cluster_dispersion_v3"
full_dir = os.path.join(og_folder, dir_name)

In [ ]:
all_files = []

# loop over files in dir
for f in os.listdir(full_dir):
    # only get mp4s
    if f.lower().endswith('.mp4'):
        all_files.append(f)

print(all_files)

---

## Pipeline functions

In [ ]:
# get the dataframe filled with all annotated frames
def get_frame_df(video_path, start_frame, end_frame):
    video = cv2.VideoCapture(video_path)
    frame_count = int(video.get(cv2.CAP_PROP_FRAME_COUNT))
    
    frames = np.array(range(start_frame, frame_count + end_frame, 1)) # assumes end_frame is negative
    
    circle_detections = []
    
    for i, frame_num  in enumerate(frames):
        img = capture_frame(video, frame_num )
        if img is not None:
            img = crop_image(img, crop_x[0], crop_y[0], crop_x[1], crop_y[1])
            grayImage = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
            
            circles = detect_circles(grayImage, radiusMin, radiusMax, canny_edge_thresh, circle_detection_thresh)
        
            if circles is not None:
                for circle in circles[0]:  # circles[0] is the list of detections
                    x, y, r = circle
                    circle_detections.append([x, y, r, frame_num])
        
            if frame_num  % 500 == 0:
                print("Done with frame", frame_num , "/", frame_count)

    return pd.DataFrame(circle_detections, columns=["x", "y", "size", "frame"])

# try to link all trajectories using the annotated frames
def get_trajs(frame_df):
    # get trajectories from the locations
    t = tp.link(frame_df, max_dist, memory=memory)
    
    # remove short trajectories
    t1 = tp.filter_stubs(t, min_length)
    
    # compare the number of particles in the unfiltered and filtered data.
    print('Before stub filtering:', t['particle'].nunique())
    print('After stub filtering:', t1['particle'].nunique())

    return t1

# smoothen the trajectories
def smooth_trajectories(df, window_length=10, polyorder=2):   
    # make sure 'frame' is only a column, not an index level
    df = df.reset_index(drop=True)
    
    # Make sure data is sorted by particle and frame
    df = df.sort_values(['particle', 'frame']).reset_index(drop=True)
    
    # group by particle and smooth each trajectory
    for pid, group in df.groupby('particle'):
        n = len(group)
        
        # adjust window_length if too short
        wl = min(window_length, n if n % 2 == 1 else n - 1)
        if wl < polyorder + 2:
            # If trajectory too short for savgol, just copy raw data
            df.loc[group.index, 'x'] = group['x']
            df.loc[group.index, 'y'] = group['y']
            continue
        
        # smooth x and y separately
        df.loc[group.index, 'x'] = savgol_filter(group['x'], wl, polyorder)
        df.loc[group.index, 'y'] = savgol_filter(group['y'], wl, polyorder)
    
    return df

# plot the trajectories for sanity check
def plot_trajectories(trajs):
    plt.figure()

    # frame size
    cropped_x_res = crop_x[1] - crop_x[0]
    cropped_y_res = crop_y[1] - crop_y[0]
    
    # make plot same shape&size as the frames
    ax = plt.gca()
    ax.set_xlim([0, cropped_x_res])
    ax.set_ylim([0, cropped_y_res])
    
    # add scale bar
    scale_bar_length_cm = 30
    
    # convert scale bar length from cm to pixels (using ratio calculated in beginning)
    scale_bar_length_pxs = scale_bar_length_cm / px_cm_ratio
    
    # location of the scale bar
    x_scale_start = 50
    y_scale_start = cropped_y_res - 50
    scale_bar_end = x_scale_start + scale_bar_length_pxs
    
    # draw a horizontal line for the scale bar
    plt.plot([x_scale_start, scale_bar_end], [y_scale_start, y_scale_start], color='black', lw=2)
    
    # add label for the scale bar
    plt.text(x_scale_start + scale_bar_length_pxs / 2, y_scale_start - 5, f'{scale_bar_length_cm} cm', 
             color='black', ha='center', va='bottom', fontsize=12)
    
    # plot traj
    tp.plot_traj(trajs, label=True)
    
    plt.show()

# save trajectory dataframe and parameters as pickle using dict
def save_as_pickle(df, video_name):
    cropped_x_res = crop_x[1] - crop_x[0]
    cropped_y_res = crop_y[1] - crop_y[0]

    file_name = video_name + ".pkl"

    save_file_path = os.path.join(save_dir, file_name)
    
    # create dict
    traj_dict = {"traj_df":df, "px_cm_ratio":px_cm_ratio, "x_res_px":cropped_x_res, "y_res_px":cropped_y_res}
    
    # save dict as pickle
    try:
        with open(save_file_path, 'wb') as file:
            pickle.dump(traj_dict, file)
            print("Pickled data:", save_file_path)
    except Exception as e:
        print(f"Error pickling dictionary: {e}")

---

## Run the full pipeline over all files in the dir

Once all paramaters are set and you ran the cells above, run this cell and everything will be processed.

In [ ]:
files_processed = 0

# loop over all files
for video_name in all_files:
    print("Start processing", video_name)

    # create path as string
    video_path = os.path.join(full_dir, video_name)
    
    # annotate all the frames
    frame_df = get_frame_df(video_path, start_frame, end_frame)

    # get the trajectories
    trajs = get_trajs(frame_df)

    # smooth the trajs if wanted
    if smoothen:
        trajs = smooth_trajectories(trajs)

    # plot if wanted
    if show_plot:
        plot_trajectories(trajs)

    # save everything to a pickle
    save_as_pickle(trajs, video_name.strip(".mp4"))

    files_processed += 1

    # progress update
    print("Done with", video_name, ",", files_processed, "/", len(all_files), "completed")